## Day 3

https://adventofcode.com/2023/day/3

### Imports

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

from aocd.models import Puzzle
from dotenv import load_dotenv

from scipy.ndimage import label

### Get data

In [2]:
load_dotenv()

puzzle = Puzzle(year=2023, day=3)
input_data_example = puzzle.examples[0].input_data
input_data = puzzle.input_data

### Part 1

In [3]:
def get_symbols(data):
    # get symbols from data
    non_symbols = [str(d) for d in np.arange(10)] + ['\n', '.']
    return [s for s in np.unique(list(data)) if s not in non_symbols]

In [4]:
def get_mask(data, symbols):
    #data = data.split("\n")
    
    # numbers to ones
    for i in np.arange(10):
        data = data.replace(str(i), "1")
    
    # dots to zeros
    data = data.replace(".", "0")
    
    # symbols to twos
    for symbol in symbols:
        data = data.replace(symbol, "2")
    
    mask = np.array([list(s) for s in data.split("\n")]).astype(int)
    return mask

In [5]:
def get_mask_b(data, symbols):
    #data = data.split("\n")
    
    # numbers to ones
    for i in np.arange(10):
        data = data.replace(str(i), "1")
    
    # dots to zeros
    data = data.replace(".", "0")
    
    # symbols to twos
    for symbol in symbols:
        if symbol == "*":
            data = data.replace(symbol, "2")
        else:
            data = data.replace(symbol, "0")
    
    mask = np.array([list(s) for s in data.split("\n")]).astype(int)
    return mask

In [6]:
def sum_data(data):
    symbols = get_symbols(data)

    # remove symbols from data
    for symbol in symbols:
        data = data.replace(symbol, ".")
    return np.sum([int(s) for line in data.split("\n") for s in line.split(".") if not s in symbols and not s in ['', ]])

In [7]:
def solution_a(data):
    symbols = get_symbols(data)
    mask = get_mask(data, symbols)

    # input data as np array
    data_np = np.array([list(s) for s in data.split("\n")])

    # use scipy label to check for un-connected parts
    labeled_array, num_features = label(mask, structure=np.ones((3, 3)))

    # loop through connected parts
    total_not_connected = 0
    for f in np.arange(1, num_features+1):
        masked_group = mask[np.where(labeled_array == f)]
        data_group = data_np[np.where(labeled_array == f)]

        # check if symbol is this group
        if 2 not in masked_group:
            part_nr = int("".join(data_group))
            total_not_connected += part_nr
    
    # get sum of all numeric values in data
    total_data = sum_data(data)
    
    # answer is total minus not connected 
    return total_data - total_not_connected

In [8]:
solution_a(input_data_example)

4361

In [9]:
print(solution_a(input_data))
puzzle.answer_a = solution_a(input_data)

coerced int64 value 532428 for 2023/03


532428


### Part 2

In [10]:
def get_gear_parts(data_np, idx_match):
    # parse the numbers out of a connected group from scipy label    
    output = []
    part_num = []
    for i in np.arange(len(idx_match[1])):
        x = idx_match[1][i]
        y = idx_match[0][i]        
        value = data_np[y, x]        
        if i > 0:
            diff_x = idx_match[1][i] - idx_match[1][i-1]
            diff_y = idx_match[0][i] - idx_match[0][i-1]
            if diff_x > 1 or diff_y != 0 or value == "*":
                output.append(part_num)
                part_num = []

        part_num.append(value)

    output.append(part_num)
    
    return [int("".join([pp.replace("*", "") for pp in p])) for p in output if len(p) > 0 and p != ["*"]]

In [11]:
def solution_b(data):
    symbols = get_symbols(data)
    mask = get_mask(data, symbols)

    # input data as np array
    data_np = np.array([list(s) for s in data.split("\n")])

    # use scipy label to check for un-connected parts
    labeled_array, num_features = label(mask, structure=np.ones((3, 3)))
    
    # loop through connected parts
    total_gear = 0
    for f in np.arange(1, num_features+1):
        idx_match = np.where(labeled_array == f)
        masked_group = mask[idx_match]
        data_group = data_np[idx_match]
       
        if "*" not in data_group:
            pass
        else:
            # work around because scipy's label is not working nicely with part two, ie. the numbers from different lines are joined together
            parts = get_gear_parts(data_np, idx_match)

            if len(parts) == 2:
                gear_ratio = np.product(parts)
                total_gear += gear_ratio
    return total_gear

In [12]:
solution_b(input_data_example)

467835

In [14]:
print(solution_b(input_data))
puzzle.answer_b = solution_b(input_data)

coerced int64 value 84051670 for 2023/03


84051670
